# Lecture 22 - Large Language Models (Part 2)

[![View notebook on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/avakanski/Fall-2025-Applied-Data-Science-with-Python/blob/main/docs/Lectures/Theme_3-Model_Engineering/Lecture_22-LLMs_Part_2/Lecture_22-LLMs_Part_2.ipynb)
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/avakanski/Fall-2025-Applied-Data-Science-with-Python/blob/main/docs/Lectures/Theme_3-Model_Engineering/Lecture_22-LLMs_Part_2/Lecture_22-LLMs_Part_2.ipynb)

<a id='top'></a>

- [22.1 Mixture of Experts](#22.1-mixture-of-experts)
  - [22.1.1 Sparsity](#22.1.1-sparsity)
  - [22.1.2 MoE Advantages and Disadvantages](#22.1.2-moe-advantages-and-disadvantages)
- [22.3 Vision-Language Models](#22.3-vision-language-models)
  - [22.3.1 VLM Architectures](#22.3.1-vlm-architectures)
  - [22.3.2 Benchmarking VLMs](#22.3.2-benchmarking-vlms)
  - [22.3.3 VLMs Importance](#22.3.3-vlms-importance)
  - [22.3.4 VLM Finetuning](#22.3.4-vlm-finetuning)



## 22.1 Mixture of Experts <a name='22.1-mixture-of-experts'></a>

**Mixture of Experts (MoE)** in LLMs refers to a technique that uses a collection of sub-networks (called "experts") that jointly perform a task, with each expert specializing in learning different aspects of the task.

MoE in LLMs has similarities with Ensemble Methods in ML (e.g., Random Forests, Gradient Boosting, Bagging Ensembles), where an ensemble of models contributes to a prediction. Differently from Ensemble Methods, MoE introduces dynamic routing of the input to different experts, as well as all experts in MoE are trained as part of a single neural architecture rather than as separate learners.

**MoE Components**

A typical MoE is shown in the figure below and consists of two key components:

- **Experts**: each expert is typically a fully-connected neural network trained on different parts or properties of the input. Each expert specializes in processing different types of inputs, such as token types, contexts, or semantic patterns.

- **Router or gating network**: analyzes the inputs and determines which tokens are sent to which experts. It acts as a coordinator that routes the inputs to the selected experts.

When MoE receives input data, it first passes the data through the gating network. The gating network analyzes the input and selects the experts to receive the data. Typically, only a small subset of experts is activated for each input. The selected experts then process the data and generate outputs, which are combined to produce the final output of the MoE.

<img src="images/moe_components.png" width="450">

*Figure: Main components of MoE.* Source: [1].




In most MoE LLMs, the dense feed-forward network (FFN) of Transformers is replaced with a set of experts. The **experts** are also feed-forward networks, but they are smaller compared to the FFN in Transformers. In general, experts can also be more complex than FFNs, and an expert can even be an MoE itself (known as hierarchical MoE).

<img src="images/transformer_vs_moe.gif" width="550">

*Figure: Transformer vs MoE.* Source: [2].


Because LLMs contain many decoder blocks in sequence, input tokens may pass through different experts across the decoder blocks, which results in diverse specialization patterns.

<img src="images/decoder_layers.gif" width="400">

*Figure: Experts in decoder layers.* Source: [2].



The **gating network (router)** is also a small FFN that acts like a multi-class classifier and outputs softmax scores over the experts. I.e., the router assigns a probability value to each expert, and determines which experts are activated for a given token.

For instance, in the figure, the router assigned the highest probability value to Expert-1, and selected it for routing the input data to that expert.

<img src="images/router.gif" width="450">

*Figure: Gating network (router).* Source: [1].



Training an MoE model involves updating both the experts and the gating network, which makes it more challenging than training traditional Transformer models. Specifically, the training can result in a poor routing strategy where the gating network activates only a small number of experts. Such imbalance means that the selected few experts can become overly specialized, while other experts can remain undertrained and underutilized. This imbalance reduces the performance of the entire model.

Consequently, *load balancing* by the router is critical for MoE performance. Several different strategies have been proposed that penalize overreliance on any one expert, and reward more even utilization of all experts.  

In MoE models, each expert learns a different specialization of the overall function. However, the experts do not learn completely separate tasks. Instead, they learn to handle different types of tokens, contexts, or transformations. For example, one expert may learn code-style token patterns, another may learn transformations helpful for math reasoning, then another may specialize in rare domain-specific words, and similar.



### 22.1.1 Sparsity<a name='22.1.1-sparsity'></a>


The main concept in MoE architectures is the presence of **sparse layers**, since only some of the experts (and therefore their neurons) are activated for each token. In the original **dense layers** in FFN of Transformers, all neurons are connected to every neuron in the preceding and following layers. In sparse layers, neurons are connected only to some of the neurons in the neighboring layers.

<img src="images/dense_sparse.png" width="450">

*Figure: Sparse versus dense networks.* Source: [3].

For example, Mixtral is a well-known MoE LLM that uses 8 experts, each containing approximately 7B parameters. Therefore, the FFN in Mixtral has 8 expert blocks. For every token, the gating network selects two of the eight experts to process the data. Hence, about 14B parameters (two experts) are active during training and inference. Note however, that the overall number of parameters in Mixtral is not exactly 8 x 7B = 56B, since some layers are shared. The total parameter count is around 47B.

The total number of parameters is generally considered a measure of the **model capacity**. The number of parameters that are used to process an individual token is a measure of the model's **computational cost**. Thus, Mixtral has a similar learning capacity to dense LLMs with 47B parameters, but has a much lower computational cost because only 14B parameters are used per token.

Note also that although 14B parameters are active per token, the entire Mixtral model with 47B parameters needs to be loaded into memory to perform inference. In other words, computational efficiency does not reduce the requirement for sufficient RAM/VRAM to load the model.

Other well-known MoE LLMs include Mixtral 8 x 22B, DeepSeek-V3 (671B parameters, with 37B active per token), Qwen3-235B (128 experts in total, 8 experts with 22B parameters active per token), Qwen1.5 (64 experts in total, 4 experts with 2.7B parameters active per token), etc. It is likely that the premier LLMs including GPT-5 and Claude 4 use some form of MoE and combine sparse and dense layers, although it is not known for sure.



### 22.1.2 MoE Advantages and Disadvantages<a name='22.1.2-moe-advantages-and-disadvantages'></a>

MoE models offer several important advantages in LLMs, as follows.

- Sparsity: Instead of activating the entire network for each input token, MoE activates only a small subset of parameters, and much of the model stays inactive for a given token.
- Efficient inference: Because only one or a few experts are used per token, fewer computations are required. This lowers inference-time compute and required memory.
- Faster pretraining: MoE LLMs are significantly faster to pretrain compared to dense models with the same total number of parameters, because each token uses only a fraction of the parameters.
- Higher quality outputs: Expert specialization improves accuracy and overall model performance.

MoE disadvantages include:

- Training instability: Sparse routing can lead to imbalanced expert usage, where a few experts get most of the traffic while others remain underutilized.
- Fine-tuning difficulty: Fine-tuning MoE models is more challenging due to the presence of both sparse and dense layers.
- Hardware complexity: MoE requires careful management of memory and compute, especially when experts are distributed across devices.
- Overfitting: Sparse models are more prone to overfitting than dense models if load balancing is insufficient.













## 22.2 Retrieval Augmenented Generation <a name='22.2-retrieval-augmenented-generation'></a>

under construction

## 22.3 Vision-Language Models <a name='22.3-vision-language-models'></a>

**Vision Language Models (VLMs)** are multimodal systems that jointly process and reason over visual (images, videos) and linguistic (text) information. By integrating the two modalities, VLMs can understand and communicate about visual content using natural language.

VLMs take both an image and its textual description as input and generate text as output. In addition, VLM variants have also been developed that can output images. In this lecture, we will focus on VLMs that output text, as the most common type.

<img src="images/vlm_structure.png" width="450">

*Figure: VLM structure.* Source: [v1].

Building datasets for VLMs requires collecting a large number of images and corresponding text, typically in the form of image captions or descriptive phrases. Several very large such datasets exist that have been essential for training modern VLMs and contain millions or billions of image-text pairs, with descriptions in English and other languages. For instance, the [LAION-5B](https://laion.ai/blog/laion-5b/) dataset has 5.8 billion image-text examples, and the [PMD (Public Model Dataset)](https://huggingface.co/datasets/facebook/pmd) contains 70 million image-text examples.

During training, a VLM learns to map visual and textual representations into a joint embedding space. This mapping allows the model to associate visual features (shapes, colors, spatial relations) with linguistic concepts, enabling it to generalize to a wide range of vision tasks and perform zero-shot inference on unseen examples.

The example below illustrates a VLM performing tasks such as object localization, segmentation, visual question answering, and image learning with instructions. The user prompts are shown on the left, and the model responses are given on the right. As demonstrated in this example, VLM models can not only interpret the semantic content of images, but can also understand spatial relations in images, such as identifying relative positions of objects, or generating segmentation masks. VLMs can also output bounding boxes of objects, and perform other spatial tasks.

<img src="images/VLM_capabilities.jpg" width="600">

*Figure: VLM prompts and responses.* Source: [v2].

In general, VLMs can perform various multimodal tasks including:

- Image and video captioning/summarization: generate context-aware descriptions of images or video frames.
- Visual question answering (VQA): answer open-ended questions based  on visual content.
- Image-based reasoning: provide explanations or logical reasoning about visual scenes.
- Multimodal dialogues: engage in conversations involving visual inputs.
- Text-to-image search: retrieve images, figures, or diagrams in documents that match a textual query.
- Image generation: generate new images based on textual prompts.

### 22.3.1 VLM Architectures<a name='22.3.1-vlm-architectures'></a>

Numerous architectures of VLMs have been proposed in recent years that employ different strategies to integrate visual and textual information. Two broad categories inlcude VLM architectures with shared multimodal embeddings and with fused multimodal embeddings, illustrated in the figure.

<img src="images/vlm_architectures.png" width="650">

*Figure: VLM architectures.* Source: [v6].


####  **VLMs with Shared Multimodal Embeddings**

VLM architectures with shared embeddings include the following main components.

**Multimodal inputs**. Input modalities in VLMs include *visual inputs* (images, PDF documents, or videos) and *textual inputs* (captions, question-answer pairs, or instructions).

**Encoding Modalities.** A *vision encoder* transforms the visual input into numerical representations, referred to as visual embeddings. The vision encoder in VLMs is commonly a variant of the Vision Transformer (ViT) architecture. A *text encoder* converts textual prompts into text embeddings. The text encoder is typically a Transformer-based encoder often pretrained on large text corpora.

**Projection into a multimodal embedding space.** The visual and textual embeddings are next projected into a shared multimodal embedding space. This step is achieved through a *projector head* (a.k.a. projector layer or just projector), which is usually implemented as a small Transformer block or a block of fully-connected layers. The projector aligns the visual and textual respresentations into a shared embedding space, allowing the model to reason simultaneously across images and language.

The shared embedding space enables VLMs to link textual concepts (e.g., cat) with corresponding visual evidence (a cat's color or location in the image), allowing reasoning across both language and vision. For instance, the model understands "cat" not just as a word, but also as a visual object in the image.

One representative model of this type of VLM architectures is CLIP (Contrastive Language-Image Pretraining). It is one of the earliest models that introduced vision-language alignment through contrastive learning. CLIP employs both a vision encoder and a text encoder, and learns a shared embedding space in which images and their textual descriptions are semantically aligned. *Contrastive learning* in CLIP is employed to associate visual and textual content by maximizing the similarity between matched image and text pairs and minimizing the similarity between mismatched ones. In the figure below, the image encoder outputs image embeddings $I_1, I_2, I_3, ..., I_N$ for each image in the batch, and the text encoder outputs text embeddings $T_1, T_2, T_3, ..., T_N$ for each corresponding text in the batch. The model computes a similarity score between each pair of image embeddings (e.g., $I_i$) and text embeddings (e.g., $T_j$) to align them into a shared embedding space $I_i\cdot T_j$. Similarity scores are calculated using the dot product (i.e., cosine similarity) $I_i\cdot T_j$ between image and text embeddings. As training progresses, the two encoders are updated so that image and text representations corresponding to similar concepts are drawn close to each other in the shared embedding space, while dissimilar concepts are pushed apart.

<img src="images/CLIP.png" width="550">

*Figure: CLIP architecture.* Source: [v5].

The pretrained vision encoder of CLIP has been widely adopted as a vision encoder component in numerous later VLM variants.

#### **VLMs with Fused Multimodal Embeddings**

Other VLM architectures fuse together image and text representations into joint multimodal embeddings to enable more advanced visual reasoning. These models typically employ a *text decoder* network from pretrained LLM, which generates a text output.

The following figure illustrates the workflow of this class of VLMs. The visual embeddings from the vision encoder are first matched by the multimodal projector to the corresponding text embeddings of the LLM. The LLM decoder receives a combined representation consisting of visual embeddings (image tokens) from the projector and text embeddings (text tokens) from the user's prompt or question. The LLM decoder serves as the text generation backbone, and using the combined image and text tokens, it generates a textual output in an autoregressive manner, one token at a time. Each new token is conditioned on previously generated tokens and on the fused multimodal embeddings.

<img src="images/vlm_fused_embeddings.png" width="400">

*Figure: Fused multimodal embeddings.* Source: [v2].

Training VLMs with fused vision and text embeddings typically involves multiple stages, as shown in the next figure. In the first stage only the multimodal projector (also referred to as fusion layer or adapter) is trained while keeping the image encoder and the LLM text decoder frozen. This is followed by a second stage that involves additional finetuning of the multimodal projector and parts of the text decoder, while keeping the image encoder frozen.

<img src="images/vlm_training.png" width="550">

*Figure: VLM training.* Source: [v2].

In addition, in some VLM architectures, the vision encoder is also finetuned in the second stage to further improve cross-modal reasoning.

Representative models of this VLM architectures are BLIP and Flamingo, which introduced cross-attention mechanism to fuse image and text embeddings (this is similar to the cross-attention module connecting the encoder and decoder in standard Transformer networks). Cross-modal attention enables direct fusion of image and text embeddings into a single multimodal representation, allowing the models to reason more efficiently over both modalities.

LLaVA (Large Language and Vision Assistant) architecture introduced a different design which replaced the cross-attention module with an MLP (Multi-Layer Perceptron) adapter for fusing multimodal embeddings. Spciefically, the MLP adapter is a small fully connected network that linearly projects vision and text tokens and then concatenates them into a fused representation that is passed to the text decoder.

These VLMs laid the foundations for the modern multimodal foundation models, such as Gemini 2.5 (Google), GPT-5 (OpenAI), Claude Opus 4 (Anthropic), Qwen-VL (Alibaba), and Mistral 3.1 (Mistral AI). The modern multimodal models typically employ MLP adapters for fusing image and text representations within a unified embedding space, and provide enhanced visual comprehension, reasoning, and dialog capabilities.

Also, many open-source VLM alternatives have made this fused-architecture functionality widely accessible to the research community, and include LLaVA (Microsoft), Qwen-VL (Alibaba), LLaMA 3.2 Vision (Meta AI), InternVL (OpenGVLab), Pixtral (Mistral AI), and others.


### 22.3.2 Benchmarking VLMs<a name='22.3.2-benchmarking-vlms'></a>

Performance of VLMs is assessed using multimodal benchmarks that evaluate models on a variety of tasks, such as reasoning, visual question answering, document comprehension, video understanding, and other tasks. Most benchmarks consist of a set of images with associated questions, often posed as multiple-choice questions. Popular benchmarks are [MMMU](https://mmmu-benchmark.github.io/), [Video-MME](https://video-mme.github.io/home_page.html), [MathVista](https://mathvista.github.io/), and [ChartQA](https://github.com/vis-nlp/ChartQA). MMMU is the most comprehensive benchmark, and contains 11.5K multimodal challenges that require knowledge and reasoning across different disciplines such as arts and engineering.

Several VLM-specific leaderboards provide comparative rankings across diverse metrics. [Vision Arena](https://lmarena.ai/leaderboard/vision) ranks models based on anonymous voting of model outputs by human preferences. [Open VLM Leaderboard](https://huggingface.co/spaces/opencompass/open_vlm_leaderboard) provides comparative ranking of VLMs according to different metrics and average scores.

### 22.3.3 VLMs Importance<a name='22.3.3-vlms-importance'></a>

Traditional computer vision (CV) models are constrained to learning from a predefined and fixed set of categories or objects for image classification or object detection (e.g., identify whether an image contains a cat or a dog). Moreover, these tasks require users to manually label a large number of images with a specific category for classification, or assign bounding boxes to multiple objects in each image for object detection, which is a tedious, time-consuming, and expensive process.

Conversely, VLMs are trained with more  detailed textual descriptions of images, where for example an image can contain cats, dogs, and other objects, as well as the text description can provide additional contextual information (e.g., the cat is sitting, the dog is running, etc.). Learning from rich natural language descriptions allows VLMs to better understand visual scenes without limiting the learning to a narrow set of visual concepts comprising a fixed number of classes and objects. Also, it eliminates the need for exhaustive manual image labeling and extends VLMs' utility beyond traditional CV tasks like classification or detection to new capabilities including reasoning, summarization, question answering, and interactive dialogue, by simply changing the text prompt.

VLMs have been applied across many domains and industries, and offer great potential to enhance visual perception. For instance, they can be used to review videos and extract insights for industrial inspection and robotics (detect faults, monitor operations, identify anomalies in real time), safety and infrastructure monitoring (recognize floods, fires, or traffic hazards), retail and logistics (track empty shelves, detect misplaced items, identify supply-chain bottlenecks), and numerous other tasks.

### 22.3.4 VLM Finetuning <a name='22.23.4-vlm-finetuning'></a>

We are excited to announce that TRL’s SFTTrainer now includes experimental support for Vision Language Models! We provide an example here of how to perform SFT on a Llava 1.5 VLM using the llava-instruct dataset which contains 260k image-conversation pairs. The dataset contains user-assistant interactions formatted as a sequence of messages. For example, each conversation is paired with an image that the user asks questions about.

## References <a name='references'></a>

1. Mixture of Experts Model(MOE) in AI: What is it and How does it work?, by Sahin Ahmed, available at [https://blog.gopenai.com/mixture-of-experts-model-moe-in-ai-what-is-it-and-how-does-it-work-b845ed38a3ab](https://blog.gopenai.com/mixture-of-experts-model-moe-in-ai-what-is-it-and-how-does-it-work-b845ed38a3ab).
2. Transformer vs. Mixture of Experts in LLMs, by Avi Chawla, avialable at [https://www.dailydoseofds.com/p/transformer-vs-mixture-of-experts-in-llms/](https://www.dailydoseofds.com/p/transformer-vs-mixture-of-experts-in-llms/).
3. What is mixture of experts?, by Dave Bergmann, available at [https://www.ibm.com/think/topics/mixture-of-experts](https://www.ibm.com/think/topics/mixture-of-experts).

v1. Vision Language Models (VLMs) Explained - GeeksForGeeks, available at [https://www.geeksforgeeks.org/artificial-intelligence/vision-language-models-vlms-explained/](https://www.geeksforgeeks.org/artificial-intelligence/vision-language-models-vlms-explained/).
v2. Vision Language Models Explained - Hugging Face Blog, by Merve,
Edward Beeching, available at [https://huggingface.co/blog/vlms](https://huggingface.co/blog/vlms).
v3. Understanding Vision-Language Models (VLMs): A Practical Guide, by Pietro Bolcato, available at [https://medium.com/@pietrobolcato/understanding-vision-language-models-vlms-a-practical-guide-8da18e9f0e0c](https://medium.com/@pietrobolcato/understanding-vision-language-models-vlms-a-practical-guide-8da18e9f0e0c).
v4. What Are Vision Language Models, by NVIDIA, available at [https://www.nvidia.com/en-us/glossary/vision-language-models/](https://www.nvidia.com/en-us/glossary/vision-language-models/).
v5. CLIP: Connecting text and images, OpenAI, available at [https://openai.com/index/clip/](https://openai.com/index/clip/).
v6. Vision-language models (VLMs) Explained (pt. 1), by Sean Trott, available at [https://seantrott.substack.com/p/vision-language-models-vlms-explained](https://seantrott.substack.com/p/vision-language-models-vlms-explained).
v7. Vision Language Models, by Rohit Bandaru, available at [https://rohitbandaru.github.io/blog/Vision-Language-Models/](https://rohitbandaru.github.io/blog/Vision-Language-Models/).